## Yield Curve Fitting

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

### 1. Data Preparation & Cleaning

In [2]:
## If cleaned data is available, read cleaned data directly
## Else, read raw data and clean it
read_raw_data = True

In [3]:
if read_raw_data:
    ## Read raw data
    yield_raw = pd.read_csv('../data/yield.csv')
    yield_raw['Date'] = pd.to_datetime(yield_raw['Date'])

    ## Cleaning raw
    ## Get rid of data before 07/30/01 since older data had no 1 MO yield data
    yield_cleaned = yield_raw.loc[yield_raw['Date']>'2001-07-30'].reset_index(drop = True)
    ## Also get rid of 2020 data since it is our target
    yield_cleaned = yield_cleaned.loc[yield_cleaned['Date']<'2020-01-01'].reset_index(drop = True)
    ## If no 30 YR yield data, use the 20 YR yield data, since they are highly related 
    ## (See Abramov et. al.)
    yield_cleaned.loc[:,'30 YR'] = [i[1] if i[2] == 0 else i[2] 
                                    for i in yield_cleaned.loc[:,['20 YR', '30 YR']].itertuples()]
    ## Get rid of 2 MO yields since it was not introduced until late 2018
    yield_cleaned = yield_cleaned.drop('2 MO', axis = 1)
    ## Save for late use
    yield_cleaned.to_csv('../data/yield_cleaned.csv', index = False)
else:
    yield_cleaned = pd.read_csv('../data/yield_cleaned.csv')
    yield_cleaned['Date'] = pd.to_datetime(yield_cleaned['Date'])

In [4]:
## Examplar cleaned data
yield_cleaned.head(5)

,Date,1 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
0,2001-07-31,3.67,3.54,3.47,3.53,3.79,4.06,4.57,4.86,5.07,5.61,5.51
1,2001-08-01,3.65,3.53,3.47,3.56,3.83,4.09,4.62,4.90,5.11,5.63,5.53
2,2001-08-02,3.65,3.53,3.46,3.57,3.89,4.17,4.69,4.97,5.17,5.68,5.57
3,2001-08-03,3.63,3.52,3.47,3.57,3.91,4.22,4.72,4.99,5.20,5.70,5.59
4,2001-08-06,3.62,3.52,3.47,3.56,3.88,4.17,4.71,4.99,5.19,5.70,5.59


In [5]:
## Cleaned data summary statistics
yield_cleaned.describe()

,1 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
count,4606.000000,4606.000000,4606.000000,4606.000000,4606.000000,4606.000000,4606.000000,4606.000000,4606.000000,4606.000000,4606.000000
mean,1.297451,1.353608,1.466077,1.577794,1.823517,2.063769,2.534783,2.903291,3.235795,3.799103,3.923356
std,1.485886,1.508155,1.536025,1.498917,1.413028,1.341612,1.224258,1.138802,1.075487,1.119692,0.985139
min,0.000000,0.000000,0.020000,0.080000,0.160000,0.280000,0.560000,0.910000,1.370000,1.690000,1.940000
25%,0.070000,0.100000,0.160000,0.270000,0.640000,0.960000,1.570000,2.000000,2.280000,2.740000,3.010000
50%,0.900000,0.950000,1.040000,1.220000,1.480000,1.650000,2.320000,2.780000,3.060000,3.880000,4.040000
75%,1.967500,2.010000,2.130000,2.350000,2.660000,2.907500,3.430000,3.850000,4.190000,4.810000,4.770000
max,5.270000,5.190000,5.330000,5.300000,5.290000,5.260000,5.230000,5.290000,5.440000,6.050000,6.050000


In [6]:
## Get delta yields (aka change in yield rate, or shock)
dyield = yield_cleaned.iloc[:-1, :].copy()
dy = yield_cleaned.iloc[:, 1:].values
dyield.iloc[:, 1:] = (np.vstack([dy, np.zeros(dy.shape[1])]) - np.vstack([np.zeros(dy.shape[1]), dy]))[1:-1]

In [7]:
## Examplar delta yield
dyield.head(5)

,Date,1 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
0,2001-07-31,-0.02,-0.01,0.00,0.03,0.04,0.03,0.05,0.04,0.04,0.02,0.02
1,2001-08-01,0.00,0.00,-0.01,0.01,0.06,0.08,0.07,0.07,0.06,0.05,0.04
2,2001-08-02,-0.02,-0.01,0.01,0.00,0.02,0.05,0.03,0.02,0.03,0.02,0.02
3,2001-08-03,-0.01,0.00,0.00,-0.01,-0.03,-0.05,-0.01,0.00,-0.01,0.00,0.00
4,2001-08-06,0.01,0.00,0.00,0.00,0.02,0.02,0.01,0.01,0.01,0.01,0.01


In [8]:
## Delta yield summary statistics
dyield.describe()

,1 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
count,4605.000000,4605.000000,4605.000000,4605.000000,4605.000000,4605.000000,4605.000000,4605.000000,4605.000000,4605.000000,4605.000000
mean,-0.000476,-0.000432,-0.000406,-0.000421,-0.000480,-0.000530,-0.000625,-0.000658,-0.000684,-0.000730,-0.000678
std,0.063741,0.046204,0.037007,0.037812,0.051412,0.055756,0.059802,0.060387,0.057272,0.054471,0.053763
min,-1.050000,-0.810000,-0.490000,-0.500000,-0.540000,-0.500000,-0.460000,-0.530000,-0.510000,-0.340000,-0.330000
25%,-0.010000,-0.010000,-0.010000,-0.010000,-0.020000,-0.030000,-0.030000,-0.040000,-0.030000,-0.030000,-0.030000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.010000,0.010000,0.010000,0.010000,0.020000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000
max,0.860000,0.760000,0.750000,0.520000,0.380000,0.370000,0.340000,0.300000,0.250000,0.260000,0.280000


In [9]:
## 12/31/2019 data to calculate shock for 2020 q1 prediction
last_date = yield_cleaned.iloc[-1, 1:].values

## Scenarios by the Fed
## baseline
baseline = np.array([1.6, 1.7, 1.8])
dbaseline = baseline - last_date[[1,6,8]]

## severly adverse
sevadv = np.array([0.1, 0.5, 0.7])
dsevadv = sevadv - last_date[[1,6,8]]

## 2. Nelson-Siegel Modeling

In [10]:
## Hyperparameter (See Abramov et. al.)
lmbda = 0.0299

## slope
def I1(T, lmbda = lmbda):
    return (1-np.exp(-lmbda*T))/(lmbda*T)

## curvature
def I2(T, lmbda = lmbda):
    return (1-np.exp(-lmbda*T))/(lmbda*T) - np.exp(-lmbda*T)

In [11]:
## level, slope, curvature matrix for training
I_train = np.array([[1, I1(3), I2(3)],
                    [1, I1(60), I2(60)],
                    [1, I1(120), I2(120)]])

## for prediction
I_pred = np.array([[1, I1(t), I2(t)] for t in [1, 3, 6, 12, 24, 36, 60, 84, 120, 240, 360]])

In [12]:
## train and predict baseline scenario
dbetas_baseline = np.linalg.inv(I_train).dot(dbaseline.T)
pred_baseline_NS = I_pred.dot(dbetas_baseline) + last_date

## train and predict severly adverse scenario
dbetas_sevadv = np.linalg.inv(I_train).dot(dsevadv.T)
pred_sevadv_NS = I_pred.dot(dbetas_sevadv) + last_date

In [13]:
## some reporting of results here

## 3. PCA

In [116]:
## Further set up
from sklearn import preprocessing
from sklearn.decomposition import PCA

In [117]:
## Matrix to perform PCA on
X = dyield.iloc[:, 1:].values.T

## Normalize before running PCA
## Cache the mean and sd first
X_mean = np.mean(X, axis = 1)
X_sd = np.std(X, axis = 1)

## Actual normalization
X_scaled = preprocessing.scale(X)
#X_scaled = X

$$X = U \Sigma V^T$$ 
$$\hat{X} = U \Sigma = XV$$
$$ note: U^T = U^{-1} $$

In [118]:
## Perform SVD on the matrix
U, Sig, Vt = np.linalg.svd(X_scaled)

In [119]:
## array to get delta PC
dbaseline_arr = np.zeros(len(last_date))
dbaseline_arr[1] = dbaseline[0]
dbaseline_arr[6] = dbaseline[1]
dbaseline_arr[8] = dbaseline[2]

dsevadv_arr = np.zeros(len(last_date))
dsevadv_arr[1] = dsevadv[0]
dsevadv_arr[6] = dsevadv[1]
dsevadv_arr[8] = dsevadv[2]

## standarize with X's parameters
dbaseline_arr = (dbaseline_arr - X_mean)/X_sd
dsevadv_arr = (dsevadv_arr - X_mean)/X_sd

## calculate delta PC
dpc_baseline = U[:3, :].dot(dbaseline_arr)
dpc_sevadv = U[:3, :].dot(dsevadv_arr)

## calculate delta y by PCA
dy_baseline = U[:, :3].dot(dpc_baseline) * X_sd + X_mean
dy_sevadv = U[:, :3].dot(dpc_sevadv) * X_sd + X_mean

## add delta y to 12/31/2019 data to get prediction
pred_baseline_PCA = dy_baseline + last_date
pred_sevadv_PCA = dy_sevadv + last_date

In [120]:
pred_baseline_PCA

array([1.4726423595449736, 1.5441301251714112, 1.5954755455255603,
       1.5859661466724104, 1.576902799955239, 1.6189026491604057,
       1.692349131125718, 1.8346534609621477, 1.9253079475313202,
       2.2561082044144944, 2.396081444837797], dtype=object)

In [121]:
pred_sevadv_PCA

array([1.5148303577516846, 1.7657496263863992, 1.7770516425453828,
       1.735213527946046, 1.6473482649872826, 1.6070144069113457,
       1.5626626671606916, 1.6456336485144905, 1.7353230099710826,
       2.0655014912464478, 2.2157740322787567], dtype=object)

## 4. ANN

In [23]:
X_train = 

SyntaxError: invalid syntax (<ipython-input-23-9e5f22140be1>, line 1)